### Decorators Application (Logger, Stacked Decorators)

In this example we're going to create a utility decorator that will log function calls (to the console, but in practice you would be writing your logs to a file (e.g. using Python's built-in logger), or to a database, etc.

In [1]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner

In [2]:
@logged
def func_1():
    pass

In [3]:
@logged
def func_2():
    pass

In [4]:
func_1()

func_1: called 2017-12-10 00:09:19.443657+00:00


In [5]:
func_2()

func_2: called 2017-12-10 00:09:19.460691+00:00


Now we may additionaly also want to time the function. We can certainly include the code to do so in our `logged` decorator, but we could also just use the `@timed` decorator we already wrote by **stacking** our decorators.

In [6]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [7]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [8]:
factorial(10)

factorial: called 2017-12-10 00:09:19.496762+00:00
factorial ran for 0.000130s


3628800

Note that the order in which we stack the decorators can make a difference!

Remember that this is because our stacked decorators essentially amounted to:

In [9]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

So in this case the `timed` decorator will be called first, followed by the `logged` decorator.

You may wonder why the printed output seems reversed. Look at how the decorators were defined - they first ran the function passed in, and **then** printed the result.

So in the above example, a simplified look at what happens in each decorator:

* `timed(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints timing
    
    
* `logged(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints log info

So, calling
`factorial = timed(logged(factorial))`

is equivalent to:

<pre>
fn = logged(factorial)
factorial = timed(fn)

factorial(n) --> call timed(fn)(n)
             --> call fn(n), then print timing
             --> call logged(original_factorial)(n), then print timing
             --> call original_factorial(n), then log, then print timing
</pre>

So as you can see, the `timed` decorator ran first, but it called the logged decorated function first, then printed the result - hence why the print output seems reversed.

In [10]:
factorial(10)

factorial: called 2017-12-10 00:09:19.525820+00:00
factorial ran for 0.000147s


3628800

But in the following case, the `logged` decorator will run first, followed by the `timed` decorator:

In [11]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

In [12]:
factorial(10)

factorial ran for 0.000015s
factorial: called 2017-12-10 00:09:19.547866+00:00


3628800

Or, using the **@** notation:

In [13]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [14]:
factorial(10)

factorial ran for 0.000016s
factorial: called 2017-12-10 00:09:19.572914+00:00


3628800

In [15]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [16]:
factorial(10)

factorial: called 2017-12-10 00:09:19.608237+00:00
factorial ran for 0.000153s


3628800

To make this clearer, let's write two very simple decorators as follows:

In [17]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

In [18]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [19]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [20]:
my_func()

running dec_1
running dec_2
running my_func


But if we change the order of the decorators:

In [21]:
@dec_2
@dec_1
def my_func():
    print('running my_func')

In [22]:
my_func()

running dec_2
running dec_1
running my_func


You may wonder whether this really matters in practice. And yes, it can.

Consider an API that contains various functions that can be called. However, endpoints are secured and can only be run by authenticated users who have some specific role(s). If they do not have the role you want to return an unauthorized error. But if they do, then you want to log that they called the endpoint.

In this case you may have one decorator that is used to check authentication and permissions (and immediately return an unauthorized error from the API if applicable), and the other to log the call. 

If you decorated it this way:

<pre>
@log
@authorize
def my_endpoint():
    pass
</pre>

then the call would always be logged.

But, in this instance:

<pre>
@authorize
@log
def my_endpoint():
    pass
</pre>

your endpoint would only get logged if the user passed the `authorize` test.